##### url = https://www.kci.go.kr/kciportal/main.kci
- __한국 학술지 인용 색인__ 홈페이지에서, 학회 정보 Crawling
- Selenium 활용
- Chrome 버전 88.0.4324.104(공식 빌드) (64비트)

In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm
import requests

from bs4 import BeautifulSoup
import re
import time

from selenium import webdriver 

chromedriver = 'chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

url = 'https://www.kci.go.kr/kciportal/main.kci'

driver.get(url)


##### 학회 정보가 있는 페이지까지 이동 ~

In [2]:
# 메인 홈페이지로부터, 크롤링하고자 하는 html 있는 페이지까지 click을 통해 이동
def click_func(xpath):
    global driver
    
    button = driver.find_element_by_xpath(xpath)
    button.click()
    time.sleep(0.1)

In [3]:
#기관정보검색칸으로 이동
institution_button = '//*[@id="gnbs"]/ul/li[4]/a'
click_func(institution_button) 

In [4]:
#학술대회검색으로 이동
conference_button = '//*[@id="tabMenu"]/li[2]/a'
click_func(conference_button)

#학술대회 종합 검색으로 이동
conference_button2 = '//*[@id="sBody"]/form/a'
click_func(conference_button2)

In [5]:
# 검색하기 버튼 클릭
search_button = '//*[@id="AKCFrm"]/div/div[3]/div/input'
click_func(search_button)

In [6]:
#(학회/대학부)중, 학회 radio 버튼 클릭  
institution_type_button = '//*[@id="insiDiv01"]'  
click_func(institution_type_button)

In [7]:
#재탐색 버튼 클릭
re_search_button = '//*[@id="conLeft"]/div/div/button/span/em'
click_func(re_search_button)

In [8]:
# 300개 list 출력
show300row_button = '//*[@id="sBody"]/div[1]/div/div[2]/div/div[3]/ul/li[1]/a'
click_func(show300row_button)

In [9]:
# 300개 list 보이도록
show_button = '//*[@id="sBody"]/div[1]/div/div[2]/div/div[4]/a'
click_func(show_button)

In [10]:
# 해당 page source를 Bs4를 활용한 parsing
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

##### 학회정보 있는 곳으로 이동 후, beatifulSoup 활용하여 Crawling

In [11]:
# 전체 데이터가 받아와질 공간
all_conference_list = []

In [12]:
# 한 페이지에 300개가 존재 => 한 페이지당 shape (300,4) 형태의 2d list로 받아옴

def append_conference_info_list(soup):
    conference_name_list = np.array([i.get('value').split(' : ')[1] for i in soup.find_all("input", attrs = {"name" : "a"})])  #학회제목
    host_institution_list = np.array([i.get('value').split(' : ')[1] for i in soup.find_all("input", attrs = {"name" : "b"})]) #개최기관
    preiod_list = np.array([i.get('value').split(' : ')[1] for i in soup.find_all("input", attrs = {"name" : "c"})]) #개최기간
    research_field_list = np.array([i.get('value').split(' : ')[1] for i in soup.find_all("input", attrs = {"name" : "d"})]) #연구 분야

    confer_info_300_list = np.column_stack([conference_name_list, host_institution_list, preiod_list, research_field_list]) # (300,4) matrx 생성

    all_conference_list.append(confer_info_300_list.tolist()) #전체 list에 append

    time.sleep(1)

In [13]:
def click_next_button():

    next_page_button = '//*[@id="sBody"]/div[2]/div/div/div/a[4]'
    click_func(next_page_button)

    #soup page source또한 초기화
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    #에레로 인해 과거페이지로 돌아가므로 , 3초간 sleep
    time.sleep(3)
    
    return soup

In [14]:
def click_behind_button():
    behind_page_button = '//*[@id="sBody"]/div[2]/div/div/div/a[3]'
    click_func(behind_page_button)

    #soup page source또한 초기화
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    return soup

In [15]:
# 먼저, 첫번째페이지 Crawling
append_conference_info_list(soup)

['2021년도 가을 총회 및 국제 학술대회' '한국화학공학회' '2021.10.27 ~ 2021.10.29' '화학공학']  length :  300


In [77]:
#전체 2~130페이지까지 존재함
#다음 페이지로 넘겨가면서, 크롤링 진행

for page in range(1, 130):
    try:
        print('page : {} 진행중'.format(page))
        soup = click_next_button()  # 다음페이지로 이동
        append_conference_info_list(soup) #이동한 페이지에서 학회정보Crawling
    
    except Exception as inst:
        print(inst)
        soup = click_behind_button()
        soup = click_next_button()

##### 크롤링한 학회 정보들 데이터프레임으로 변환

In [86]:
import itertools

# itertools 함수를 활용하여 , 2D List => 1D list
list_2D_to_1D = list(itertools.chain.from_iterable(all_conference_list))
df = pd.DataFrame(list_2D_to_1D)

df.columns = ['학회명', '개최기관명', '개최기간', '연구분야']
df['year'] = df['개최기간'].str[:4].astype(int)

df.tail(3)

,학회명,개최기관명,개최기간,연구분야,year
38697,2007년 춘계 학술대회,한국운동재활학회,2007.06.08 ~ 2007.06.08,사회과학,2007
38698,2007년도 제23차 춘계 학술대회 및 연수교육,대한흉부외과학회,2007.06.07 ~ 2007.06.08,흉부외과학,2007
38699,2007년도 춘계 학술대회,한국실험동물학회,2007.06.07 ~ 2007.06.08,의약학,2007


In [95]:
# 연도별 학술 개최현황
df[df['year'] >= 2011]['year'].value_counts().sort_index()

2011    2936
2012    2967
2013    3069
2014    3151
2015    3150
2016    3220
2017    3051
2018    2748
2019    2182
2020     455
2021      11
Name: year, dtype: int64